In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymysql

In [2]:
url='https://movie.naver.com/movie/running/current.naver'
base='https://movie.naver.com/'
res=requests.get(url).content
soup=bs(res,'html.parser')
#print(soup)
movies=soup.select('div.lst_wrap > ul > li') # 해당하는 선택자의 li태그들을 리스트형태로 저장
#print(movies[0].text)
movie_list=[]
for movie in movies:
    title=movie.select('dl > dt > a')[0].text # 해당 선택자를 리스트형태로 저장하여 첫번째에 해당하는 텍스트를 출력
    reated=movie.select_one('dl > dt > span').text
    point=movie.select_one('dl > dd.star > dl > dd > div > a > span.num').text
    #print(title,reated,point)
    info=movie.select_one('dl.info_txt1 > dd').text
    #arr_info=info.split('|')
    #print(arr_info)
    arr_info=info.replace('\t','').replace('\n\n','').replace('\r\n','').split('|')
    if len(arr_info)<3:
        arr_info.insert(0,'장르정보 없음')
    genre=arr_info[0]
    running=arr_info[1].replace('분','')
    release_date=arr_info[2].replace('개봉','')
    #print(genre,running,release_date)
    director=movie.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(4) > span > a').text
    #print(director)
    actors=movie.select('dl > dd:nth-child(3) > dl > dd:nth-child(6) > span > a')
    #print(actors)
    actorstr=''
    for actor in actors:
        actorstr+=actor.text+','
    actors=actorstr.strip(',')
    #movie_list.append((title,reated,point,genre,running,release_date,director,actorstr))
    # 상세페이지 
    href=movie.select('dl > dt > a')[0]['href']
    url2=base+href
    res2=requests.get(url2)
    soup2=bs(res2.text,'html.parser')
    story=soup2.select('#content > div.article > div.section_group.section_group_frst > div:nth-child(1) > div > div.story_area > p')[0].text.replace('\r','')
    #print(story)
    
    temp=soup2.select('#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph')
    if len(temp)>0:
        temp=temp[0].text.replace('이상','').replace('\n','').split('대')
        temp=temp[:-1]
        temp=' '.join(temp)
    else:
        temp='선호도 정보 없음'
    #print(temp)
    movie_list.append((title,reated,point,genre,running,release_date,director,actorstr,story,temp))

In [3]:
df=pd.DataFrame(movie_list,columns=['title','rated','point','genre','running','relase_date','director','actors','stroy','temp'])
df

,title,rated,point,genre,running,relase_date,director,actors,stroy,temp
0,내 친한 친구의 아침식사,12세 관람가,7.06,멜로/로맨스,119,2022.11.10,두정철,"에릭 추,이목,송백위,하사정,","세상 종말이 와도 먹는 것을 멈출 수 없는 귀요미 먹방 요정, 웨이신 좋아하는 ...",1%10 24%20 41%30 24%40 10%50
1,극장판 소드 아트 온라인 -프로그레시브- 짙은 어둠의 스케르초,12세 관람가,7.22,"애니메이션, 액션, 판타지",101,2022.11.10,카와노 아야코,,가상의 게임 속 몬스터들에 맞서 싸우는 캐릭터들의 활약과 모험을 그린 애니메이션 영화,0%10 75%20 25%30 0%40 0%50
2,첫번째 아이,12세 관람가,8.10,드라마,93,2022.11.10,허정재,"박하선,오동민,오민애,공성하,임형국,",첫 아이가 태어나고 일 년 후 ‘정아’는 회사에 복직한다. 사회초년생 ‘지현’은 ...,0%10 25%20 75%30 0%40 0%50
3,같은 속옷을 입는 두 여자,15세 관람가,8.51,드라마,140,2022.11.10,김세인,"임지호,양말복,","여느 날과 다름없이 다투던 중, 마트 주차장에서 ‘수경’이 탄 차가 ‘이정’을 덮치...",0%10 29%20 43%30 14%40 14%50
4,피가로~피가로~피가로,12세 관람가,8.50,"코미디, 드라마, 멜로/로맨스",105,2022.11.10,벤 르윈,"다니엘 맥도널드,휴 스키너,조안나 럼리,",런던의 잘나가는 펀드 매니저 밀리는 오래 전부터 꿈꿔왔던 오페라 가수에 도전하기 위...,0%10 0%20 50%30 0%40 50%50
...,...,...,...,...,...,...,...,...,...,...
98,금지된 장난,12세 관람가,8.88,"드라마, 전쟁",102,1992.09.26,르네 클레망,"조르주 푸줄리,브리지트 포시,",1940년 6월 남프랑스의 농촌 마을에 파리에서 피난오다 공습으로 부모를 잃고 죽은...,0%10 0%20 100%30 0%40 0%50
99,초대받은 사람들,전체 관람가,7.00,장르정보 없음,110,1981.10.24,최하원,"이영하,원미경,박암,남성훈,",제20회(81년)대종상최우수작품상 수상 1784년 천주교가 들어오고 이승훈은 정약종...,선호도 정보 없음
100,모정,12세 관람가,8.92,"드라마, 멜로/로맨스",102,1972.12.09,헨리 킹,"윌리엄 홀든,제니퍼 존스,","1949년, 홍콩은 내전으로 얼룩진 중국본토로부터 쏟아져 들어온 많은 피난민들이 머...",선호도 정보 없음
101,욕망이라는 이름의 전차,12세 관람가,9.25,드라마,122,1957.04.19,엘리아 카잔,"비비안 리,말론 브란도,",블랑쉬(Blanche: 비비안 리 분)는 욕망이라는 이름의 전차를 타고 미국 남부 ...,선호도 정보 없음


In [4]:
conn=pymysql.connect(host='localhost',user='jhs',password='1234',db='pydb',charset='utf8')
cursor=conn.cursor()
sql="insert into movie_list(title,rated,point,genre,running,relase_date,director,actors,story,temp) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql,movie_list)
conn.commit()
conn.close()

OperationalError: (1045, "Access denied for user 'jhs'@'localhost' (using password: YES)")